#####Criar uma tabela de produtos unindo categorias, marcas e quantidade total de estoque em todas as lojas

In [0]:
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
%sql
-- cte, grava uma consulta sql dentro de uma variavel

with stock as (
  select
  product_id, sum(quantity) as total_stock
  from tmp_bronze_stocks
  group by product_id
)


In [0]:
%sql
describe tmp_bronze_stocks

In [0]:
%sql

with stock as (
  select
  product_id, sum(quantity) as total_stock
  from tmp_bronze_stocks
  group by product_id
)

select
  p.product_id,
  p.product_name,
  --,p.brand_id
  b.brand_name,
  --,b.brand_id as brand_id_brand
  --,p.category_id
  --,c.category_id as category_id_categoriat
  c.category_name,
  p.model_year,
  p.list_price,
  s.total_stock
from
  tmp_bronze_products as p
    left join tmp_bronze_categories as c
      on p.category_id = c.category_id
    left join tmp_bronze_brands as b
      on p.brand_id = b.brand_id
    left join stock as s 
      on p.product_id = s.product_id


In [0]:
# cria tábela temporária após a tratativa

df_product_silver = spark.sql(
"""
    with stock as (
  select
  product_id, sum(quantity) as total_stock
  from tmp_bronze_stocks
  group by product_id
)

select
  p.product_id,
  p.product_name,
  --,p.brand_id
  b.brand_name,
  --,b.brand_id as brand_id_brand
  --,p.category_id
  --,c.category_id as category_id_categoriat
  c.category_name,
  p.model_year,
  p.list_price,
  s.total_stock
from
  tmp_bronze_products as p
    left join tmp_bronze_categories as c on p.category_id = c.category_id
    left join tmp_bronze_brands     as b on p.brand_id = b.brand_id
    left join stock                 as s on p.product_id = s.product_id
""")

#salva em delta
df_product_silver.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{silver_path}/product')

#####Criar tabela

In [0]:
%sql
CREATE TABLE bikestore.logistics.product_silver
AS
SELECT *
FROM delta.`abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/product`;

In [0]:
%sql
SELECT * FROM bikestore.logistics.product_silver